In [1]:
import pandas as pd
from modeling.utils import load_config
from bertopic import BERTopic
import numpy as np
import torch
config = load_config()


2024-06-11 06:23:54.334008: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 06:23:54.482169: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 06:23:55.174063: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("../data/processed/BERTopic-corp.csv", index_col=0)
df.head()

,id,text,BERTopics
0,7665777-1,early physical therapist intervention patient ...,-1
1,7665777-2,early physical therapist intervention patient ...,-1
2,7665777-3,early physical therapist intervention patient ...,-1
3,7665777-4,early physical therapist intervention patient ...,-1
4,7665777-5,early physical therapist intervention patient ...,-1


In [3]:
ids = list(df.index)
ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [4]:
# docs = df.text.to_numpy().flatten()

In [5]:
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer("Charangan/MedBERT")

In [6]:
# BERT = BERTopic.load("../BERTModels/my_model.pkl", embedding_model=embedding_model)

In [7]:
# BERT.get_topic_info()

In [8]:
# import torch, gc
# gc.collect()
# torch.cuda.empty_cache()

In [9]:
# embeddings = embedding_model.encode(docs, batch_size=64, convert_to_numpy=True, show_progress_bar=True)

In [10]:
# np.save("../data/processed/Document_Embeddings.npy", embeddings)

In [11]:
embeddings = np.load("../data/processed/Document_Embeddings.npy")

In [12]:
embeddings

array([[ 0.07162705, -0.5124514 , -0.35952565, ..., -0.27501535,
        -0.3400203 ,  0.31884128],
       [ 0.07464305, -0.41961193, -0.17686662, ..., -0.3495943 ,
        -0.24562414,  0.35183027],
       [ 0.11793648, -0.518762  , -0.23346135, ..., -0.19809987,
        -0.4267902 ,  0.19690108],
       ...,
       [ 0.25113267, -0.37451935, -0.11767858, ..., -0.15912151,
        -0.2908483 ,  0.49034804],
       [ 0.05632647, -0.07859585, -0.23251684, ...,  0.01553675,
        -0.0969719 ,  0.36347044],
       [ 0.09254633, -0.32474533, -0.27699035, ..., -0.04035068,
        -0.24767956,  0.67027795]], dtype=float32)

# QDrant

In [13]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")

In [14]:
client.create_collection(
    collection_name="PMC-Patient",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `PMC-Patient` already exists!"},"time":0.000038201}'

In [ ]:
client.upload_collection(
    collection_name="PMC-Patient",
    vectors=embeddings,
    ids=ids,
    # Vector ids will be assigned automatically
    batch_size=128,  # How many vectors will be uploaded in a single request?
)